In [41]:
import tensorflow as tf
num_classes = 10
img_rows , img_cols = 28,28
num_channels = 1
input_shape = (img_rows, img_cols, num_channels)

(X_train,y_train),(X_test,y_test) = tf.keras.datasets.mnist.load_data()
X_train, x_test =  X_train/255.0,  X_test/255.0

In [53]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(128,activation='relu'))
model.add(tf.keras.layers.Dense(num_classes,activation='softmax'))

In [54]:
model.compile(optimizer='sgd',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [44]:
model.fit(X_train,y_train,epochs=5,verbose=1,validation_data=(X_test,y_test))

Epoch 1/5
1875/1875 [==============================] - 3s 1ms/step - loss: 0.6325 - accuracy: 0.8424 - val_loss: 35.7867 - val_accuracy: 0.9052
Epoch 2/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.3353 - accuracy: 0.9071 - val_loss: 35.5186 - val_accuracy: 0.9185
Epoch 3/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.2877 - accuracy: 0.9192 - val_loss: 34.5303 - val_accuracy: 0.9223
Epoch 4/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.2576 - accuracy: 0.9274 - val_loss: 31.1444 - val_accuracy: 0.9298
Epoch 5/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.2341 - accuracy: 0.9339 - val_loss: 29.2213 - val_accuracy: 0.9344


In [45]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_2 (Flatten)         (None, 784)               0         
                                                                 
 dense_4 (Dense)             (None, 128)               100480    
                                                                 
 dense_5 (Dense)             (None, 10)                1290      
                                                                 
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________


In [10]:
import tensorflow as tf

a = tf.constant([1,2,3])
b = tf.constant([0,0,1])
c = tf.add(a,b)

print(c)

tf.Tensor([1 2 4], shape=(3,), dtype=int32)


## 그래프 생성하기

In [11]:
def compute(a,b,c):
    d = a*b+c
    e = a*b*c
    return d,e

In [22]:
@tf.function
def compute(a,b,c):
    d = a*b+c
    e = a*b*c
    print('Creating Graph !')
    tf.print('Creating Graph !!')
    return d,e

In [23]:
result = compute(3,4,5)

Creating Graph !
Creating Graph !!


In [25]:
result1 = compute(3,4,5)

Creating Graph !!


In [24]:
result[0].numpy()

17

## Gradient Tape : 자동미분

In [13]:
A,b = tf.constant(3.0),tf.constant(6.0)
X = tf.Variable(20.0)
loss = tf.math.abs(A*X - b)
loss

<tf.Tensor: shape=(), dtype=float32, numpy=54.0>

In [15]:
def train_sep():
    with tf.GradientTape() as tape:
        loss = tf.math.abs(A*X-b)
    dX = tape.gradient(loss,X)
    print('X = {:.2f}, dX = {:2f}'.format(X.numpy(),dX))
    X.assign(X-dX)

for i in range(7):
    train_sep()

X = 20.00, dX = 3.000000
X = 17.00, dX = 3.000000
X = 14.00, dX = 3.000000
X = 11.00, dX = 3.000000
X = 8.00, dX = 3.000000
X = 5.00, dX = 3.000000
X = 2.00, dX = 0.000000


## FunctionalAPI

In [26]:
model_input = tf.keras.layers.Input(shape=input_shape)
output = tf.keras.layers.Flatten()(model_input)
output = tf.keras.layers.Dense(128,activation='relu')(output)
output = tf.keras.layers.Dense(num_classes,activation='softmax')(output)
model = tf.keras.Model(model_input,output)

## Tensorflow2의 변수

In [28]:
print([variable.name for variable in model.variables])

['dense_2/kernel:0', 'dense_2/bias:0', 'dense_3/kernel:0', 'dense_3/bias:0']


In [29]:
a = tf.Variable(3,name='my_var')
print(a)

<tf.Variable 'my_var:0' shape=() dtype=int32, numpy=3>


In [30]:
a.numpy()

3

In [33]:
a.name

'my_var:0'

In [34]:
a.shape

TensorShape([])

In [36]:
a.assign(a+1)
print(a.numpy())

4


In [37]:
b = a+1
print(b)

tf.Tensor(5, shape=(), dtype=int32)


## 사전제작된 estimator

## 맞춤형 estimator 훈련시키기

In [46]:
estimator = tf.keras.estimator.model_to_estimator(model,model_dir='./estimator_dir')

INFO:tensorflow:Using default config.
INFO:tensorflow:Using the Keras model provided.


/Users/yughyeonjun/opt/anaconda3/envs/Six/lib/python3.7/site-packages/keras/backend.py:450: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn('`tf.keras.backend.set_learning_phase` is deprecated and '
2022-07-09 21:44:11.462619: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:354] MLIR V1 optimization pass is not enabled


INFO:tensorflow:Using config: {'_model_dir': './estimator_dir', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [50]:
batch_size = 32
def train_input_fn():
    train_dataset = tf.data.Dataset.from_tensor_slices((X_train,y_train))
    train_dataset = train_dataset.batch(batch_size).repeat()
    return train_dataset

In [51]:
estimator.train(train_input_fn,steps=len(X_train)//batch_size)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Warm-starting with WarmStartSettings: WarmStartSettings(ckpt_to_initialize_from='./estimator_dir/keras/keras_model.ckpt', vars_to_warm_start='.*', var_name_to_vocab_info={}, var_name_to_prev_var_name={})
INFO:tensorflow:Warm-starting from: ./estimator_dir/keras/keras_model.ckpt
INFO:tensorflow:Warm-starting variables only in TRAINABLE_VARIABLES.
INFO:tensorflow:Warm-started 4 variables.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into ./estimator_dir/model.ckpt.
INFO:tensorflow:./estimator_dir/model.ckpt-0.index
INFO:tensorflow:0
INFO:tensorflow:./estimator_dir/model.ckpt-0.data-00000-of-00002
INFO:tensorflow:400
INFO:tensorflow:./estimator_dir/model.ckpt-0.data-0000

## Callbacks

In [55]:
callbacks = [tf.keras.callbacks.TensorBoard('./logs_keras')]
model.fit(X_train,y_train,epochs=5,verbose=1,validation_data=(X_test,y_test),callbacks=callbacks)

Epoch 1/5
1875/1875 [==============================] - 3s 1ms/step - loss: 0.6453 - accuracy: 0.8379 - val_loss: 37.7588 - val_accuracy: 0.8969
Epoch 2/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.3380 - accuracy: 0.9048 - val_loss: 35.6934 - val_accuracy: 0.9160
Epoch 3/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.2911 - accuracy: 0.9180 - val_loss: 34.0957 - val_accuracy: 0.9240
Epoch 4/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.2604 - accuracy: 0.9265 - val_loss: 31.9073 - val_accuracy: 0.9301
Epoch 5/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.2371 - accuracy: 0.9336 - val_loss: 29.8036 - val_accuracy: 0.9331


In [58]:
!tensorboard --logdir ./logs_keras


NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorflow/tensorboard/issues/4784

Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.9.1 at http://localhost:6006/ (Press CTRL+C to quit)
^C


In [59]:
writer = tf.summary.create_file_writer('./model_logs')
with writer.as_default():
    tf.summary.scalar('custom_log',10,step=3)

In [60]:
accuracy = tf.keras.metrics.Accuracy()
ground_truth, predictions = [1,0,1],[1,0,0] #실제로 이것은 모델에서부터 나온다.
accuracy.update_state(ground_truth,predictions)
tf.summary.scalar('accuracy',accuracy.result(),step=4)

<tf.Tensor: shape=(), dtype=bool, numpy=False>

In [ ]:
!tensorboard --logdir ./model_logs


NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorflow/tensorboard/issues/4784

Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.9.1 at http://localhost:6006/ (Press CTRL+C to quit)
